In [1]:
import pandas as pd

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 8.2MB 5.3MB/s 


In [3]:
import pymorphy2
import math

In [118]:
def change(old, new):
  morph = pymorphy2.MorphAnalyzer()
  if (len(new.split(' ')) == 1):
    old_parse = morph.parse(old)[0]
    new_parse = morph.parse(new)[0]
    if (old_parse.tag.POS == 'NOUN' and new_parse.tag.POS == 'NOUN'):
      pad = old_parse.tag.case
      new = new_parse.inflect({pad}).word
      return new
    if (old_parse.tag.POS == 'INFN' and new_parse.tag.POS == 'INFN'):
      return new
    if (old_parse.tag.POS == 'INFN' and new_parse.tag.POS == 'VERB'):
      return new_parse.normal_form
    if (old_parse.tag.POS == 'VERB' and new_parse.tag.POS == 'INFN'):
      time = old_parse.tag.tense
      number = old_parse.tag.number
      if (number == 'plur'):
        new = new_parse.inflect({time, number}).word
        return new
      gender = old_parse.tag.gender
      new = new_parse.inflect({time, number, gender}).word
      return new
    if (old_parse.tag.POS == 'VERB' and new_parse.tag.POS == 'VERB'):
      time = old_parse.tag.tense
      number = old_parse.tag.number
      if (number == 'plur'):
        new = new_parse.inflect({time, number}).word
        return new
      person = old_parse.tag.person
      if (time == 'past'):
        gender = old_parse.tag.gender
        new = new_parse.inflect({time, number, gender, person}).word
        return new
      new = new_parse.inflect({time, number, person}).word
      return new
  else:
    old_parse = morph.parse(old.split(' ')[0])[0]
    if (old_parse.tag.POS == 'VERB'):
      verb_positions = []
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'VERB' or word_parse.tag.POS == 'INFN'):
          verb_positions.append(i)
      for k in verb_positions:
        new_parse = morph.parse(new.split(' ')[k])[0]
        time = old_parse.tag.tense
        number = old_parse.tag.number
        if (number == 'plur'):
          new_change_word = new_parse.inflect({time, number}).word
          split_list = new.split(' ')
          split_list[k] = new_change_word
          new = ' '.join(split_list)
          continue
        if (time == 'past'):
          gender = old_parse.tag.gender
          new_change_word = new_parse.inflect({time, number, gender}).word
          split_list = new.split(' ')
          split_list[k] = new_change_word
          new = ' '.join(split_list)
          continue
        person = old_parse.tag.person
        new_change_word = new_parse.inflect({time, number, person}).word
        split_list = new.split(' ')
        split_list[k] = new_change_word
        new = ' '.join(split_list)
      return new
    if (old_parse.tag.POS == 'INFN'):
      k = 0
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'INFN'):
          return new
        if (word_parse.tag.POS == 'VERB'):
          k = i
          break
      new_parse = morph.parse(new.split(' ')[k])[0]
      new_change_word = new_parse.normal_form
      split_list = new.split(' ')
      split_list[k] = new_change_word
      new = ' '.join(split_list)
      return new
    if (old_parse.tag.POS == 'NOUN'):
      noun_positions = []
      for i in range (len(new.split(' '))):
        word = new.split(' ')[i]
        word_parse = morph.parse(word)[0]
        if (word_parse.tag.POS == 'NOUN'):
          noun_positions.append(i)
      if (new.split(' ')[1] == 'и' or new.split(' ')[1] == 'или'):
        for k in noun_positions:
          pad = old_parse.tag.case
          number = old_parse.tag.number
          new_parse = morph.parse(new.split(' ')[k])[0]
          new_change_word = new_parse.inflect({pad, number}).word
          split_list = new.split(' ')
          split_list[k] = new_change_word
          new = ' '.join(split_list)
        return new
      else:
        pad = old_parse.tag.case
        number = old_parse.tag.number
        new_parse = morph.parse(new.split(' ')[0])[0]
        new_change_word = new_parse.inflect({pad, number}).word
        split_list = new.split(' ')
        split_list[0] = new_change_word
        new = ' '.join(split_list)
      return new

In [109]:
change('Предоставлению со стороны субъектов отчетности о выбросах ПГ', 'Оценка альтернатив блоков ЖЦ ВЭС')

['П']
here
datv


'оценке альтернатив блоков ЖЦ ВЭС'

In [5]:
class Tree():
    def __init__(self,root):
        self.root = root
        self.children = []
        self.Nodes = []

    def addNode(self,obj):
        self.children.append(obj)

    def changeAll(self):
      for child in self.children:
        child.changePattern()
      for child in self.children:
        if child.changeChildNodes() != None:
          child.getChildNodes()
        
    def getAllNodes(self):
        self.Nodes.append(self.root)
        for child in self.children:
            self.Nodes.append(child.data)
        for child in self.children:
            if child.getChildNodes(self.Nodes) != None:
                child.getChildNodes(self.Nodes)
        print(*self.Nodes, sep = "\n")
        print('Tree Size:' + str(len(self.Nodes)))

In [6]:
class Node():
    def __init__(self, data, template):
        self.data = data
        self.template = template
        self.arg = self.findArg(data)
        self.children = []

    def findArg(self, data):
      start = finish = 0
      for i in range(len(data)):
        if data[i] == '$' and start == 0:
          start = i + 1
          continue
        if data[i] == '$' and start != 0 and finish == 0:
          finish = i
          break
      return data[start:finish]

    def addNode(self,obj):
        self.children.append(obj)

    def changePattern(self):
      res = change(self.arg, self.template)
      start = finish = 0
      for i in range(len(self.data)):
        if self.data[i] == '$' and start == 0:
          start = i + 1
          continue
        if self.data[i] == '$' and start != 0 and finish == 0:
          finish = i
          break
      final = self.data[:start] + res + self.data[finish:]
      self.data = final
      #print(self.data)

    def getChildNodes(self,Tree):
        for child in self.children:
            if child.children:
                child.getChildNodes(Tree)
                Tree.append(child.data)
            else:
                Tree.append(child.data)

    def changeChildNodes(self):
        for child in self.children:
            if child.children:
                child.getChildNodes()
                child.changePattern()
            else:
                child.changePattern()

In [119]:
MyTree = Tree('Новая схема')
MyTree.addNode(Node('Даю $апельсины и яблоки$ другу', 'картошка и морковь'))
MyTree.addNode(Node('Смотрю с утра $новости$', 'фильм'))
MyTree.addNode(Node('Перейти к  $Предоставлению со стороны субъектов отчетности о выбросах ПГ$', 'Оценка альтернатив блоков ЖЦ ВЭС'))
MyTree.children[0].addNode(Node('$Составление перечней альтернативных процессов ЖЦ ВЭС и их описаний$', 'Формированиями требований к итоговым процессам ЖЦ ВЭС от потребляющих их выход процессов'))
MyTree.changeAll()
MyTree.getAllNodes()

Новая схема
Даю $картошки и моркови$ другу
Смотрю с утра $фильм$
Перейти к  $оценке альтернатив блоков ЖЦ ВЭС$
$формирование требований к итоговым процессам ЖЦ ВЭС от потребляющих их выход процессов$
Tree Size:5
